In [1]:
from datasets import load_dataset

/data/pufanyi/anaconda3/anacondabin/envs/live_bench/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = load_dataset("lmms-lab/LiveBench", "2024-09", split="test")

Generating test split: 100%|██████████| 313/313 [00:00<00:00, 324.63 examples/s]


In [3]:
from tqdm import tqdm

subsets = {}
for item in tqdm(data, total=len(data)):
    subset = eval(item["website"])["subject"]
    if subset not in subsets:
        subsets[subset] = 0
    subsets[subset] += 1

100%|██████████| 313/313 [00:25<00:00, 12.06it/s]


In [4]:
subsets

{'entertainment': 34,
 'sports': 44,
 'technology': 44,
 'finance': 43,
 'environment': 31,
 'politics': 40,
 'science': 38,
 'artandculture': 39}

In [5]:
sum(subsets.values()) / len(subsets)

34.75

In [12]:
df = data.to_pandas()

In [9]:
def update(row):
    if row["subtask"] == "Evaluative Questions":
        row["subtask"] = "Divergent Thinking"

In [14]:
df.apply(update, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
308    None
309    None
310    None
311    None
312    None
Length: 313, dtype: object

In [20]:
df = df[df["score"] > 6]

In [21]:
len(df)

251

In [16]:
from tqdm import tqdm

subsets = {}
for idx, item in tqdm(df.iterrows(), total=len(df)):
    subset = eval(item["website"])["subject"]
    if subset not in subsets:
        subsets[subset] = 0
    subsets[subset] += 1

100%|██████████| 313/313 [00:00<00:00, 11041.63it/s]


In [38]:
final_data = {}
for idx, item in tqdm(df.iterrows(), total=len(df)):
    subtask = item["subtask"]
    if subtask == "Evaluative Questions":
        subtask = "Divergent Thinking"
        item["subtask"] = subtask
    if subtask not in final_data:
        final_data[subtask] = []
    final_data[subtask].append(item)

100%|██████████| 312/312 [00:00<00:00, 10684.90it/s]


In [39]:
del final_data["Further Insights"]

KeyError: 'Further Insights'

In [40]:
final_data.keys()

dict_keys(['Concrete Recognition', 'Analytical Questions', 'Divergent Thinking', 'Real-world Assistance'])

In [41]:
from random import shuffle


for key, value in final_data.items():
    shuffle(value)
    value = sorted(value, key=lambda x: x["score"])
    value = list(reversed(value))[:50]
    final_data[key] = value

In [42]:
import pandas as pd

final_df = pd.concat([pd.DataFrame(value) for value in final_data.values()])

In [8]:
subsets

{'entertainment': 34,
 'sports': 44,
 'technology': 44,
 'finance': 43,
 'environment': 31,
 'politics': 40,
 'science': 38,
 'artandculture': 39}

In [43]:
from tqdm import tqdm

subsets = {}
for idx, item in tqdm(final_df.iterrows(), total=len(final_df)):
    subset = item["subtask"]
    if subset not in subsets:
        subsets[subset] = 0
    subsets[subset] += 1

100%|██████████| 200/200 [00:00<00:00, 11090.61it/s]


In [44]:
len(final_df)

200

In [45]:
subsets

{'Concrete Recognition': 50,
 'Analytical Questions': 50,
 'Divergent Thinking': 50,
 'Real-world Assistance': 50}

In [36]:
df = df[df["subtask"] != "Further Insights"]

In [49]:
from datasets import Dataset

new_data = Dataset.from_pandas(final_df, preserve_index=False, features=data.features)

In [50]:
new_data[0]

{'id': 41,
 'images': [<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1017x1317>],
 'website': "{'name': 'technology_3.png', 'subject': 'technology'}",
 'question': "Identify each vulnerability's CVE identifier along with the detailed descriptions provided in the image.",
 'answer': "{'1': {'CVE Identifier': 'CVE-2024-29824', 'Description': 'Ivanti Endpoint Manager (EPM) SQL Injection Vulnerability: Ivanti Endpoint Manager (EPM) contains a SQL injection vulnerability in Core server that allows an unauthenticated attacker within the same network to execute arbitrary code.'}, '2': {'CVE Identifier': 'CVE-2023-25280', 'Description': 'D-Link DIR-820 Router OS Command Injection Vulnerability: D-Link DIR-820 routers contain an OS command injection vulnerability that allows a remote, unauthenticated attacker to escalate privileges to root via a crafted payload with the ping_addr parameter to ping.ccp.'}, '3': {'CVE Identifier': 'CVE-2020-15415', 'Description': 'DrayTek Multiple Vigor R

In [51]:
new_data

Dataset({
    features: ['id', 'images', 'website', 'question', 'answer', 'criteria', 'subtask', 'data_generator', 'checker', 'date_time', 'screen_shoter', 'screen_size', 'score', 'reason', 'scorer_name'],
    num_rows: 200
})

In [52]:
new_data.push_to_hub("lmms-lab/LiveBench", "2024-09", split="test")

Uploading the dataset shards: 100%|██████████| 1/1 [00:10<00:00, 10.69s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/lmms-lab/LiveBench/commit/91ed85da9ab4e4eb3babf7c5e39f506a33e62ba1', commit_message='Upload dataset', commit_description='', oid='91ed85da9ab4e4eb3babf7c5e39f506a33e62ba1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/lmms-lab/LiveBench', endpoint='https://huggingface.co', repo_type='dataset', repo_id='lmms-lab/LiveBench'), pr_revision=None, pr_num=None)

In [59]:
new_data = load_dataset("lmms-lab/LiveBench", "2024-09", split="test")
df = new_data.to_pandas()

Generating test split: 100%|██████████| 200/200 [00:00<00:00, 274.48 examples/s]


In [62]:
def get_subset(row):
    return eval(row)["subject"]


df["website"] = df["website"].apply(get_subset)

In [64]:
df.iloc[0]["website"]

'technology'

In [65]:
new_data = Dataset.from_pandas(df, preserve_index=False, features=new_data.features)
new_data.push_to_hub("lmms-lab/LiveBench", "2024-09", split="test")

Uploading the dataset shards: 100%|██████████| 1/1 [00:08<00:00,  8.85s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/lmms-lab/LiveBench/commit/c83aaca619b54f3115ea7348139ea0ee6fb139c6', commit_message='Upload dataset', commit_description='', oid='c83aaca619b54f3115ea7348139ea0ee6fb139c6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/lmms-lab/LiveBench', endpoint='https://huggingface.co', repo_type='dataset', repo_id='lmms-lab/LiveBench'), pr_revision=None, pr_num=None)

In [58]:
from PIL import Image
import io


def get_data():
    for index, item in df.iterrows():
        new_item = item.to_dict()
        new_item["subset"] = eval(item["website"])["subject"]
        del new_item["website"]
        new_item["id"] = index
        image_bytes = new_item["images"][0]["bytes"]
        image = Image.open(io.BytesIO(image_bytes))
        new_item["images"] = [image]
        yield new_item


dataset = Dataset.from_generator(get_data)

Generating train split: 200 examples [00:01, 132.53 examples/s]


KeyboardInterrupt: 

In [56]:
dataset.push_to_hub("lmms-lab/LiveBench", "2024-09", split="test")

Uploading the dataset shards: 100%|██████████| 1/1 [00:16<00:00, 16.91s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/lmms-lab/LiveBench/commit/a25ad8c1a882b28a838d6ca3afd26cd29beac153', commit_message='Upload dataset', commit_description='', oid='a25ad8c1a882b28a838d6ca3afd26cd29beac153', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/lmms-lab/LiveBench', endpoint='https://huggingface.co', repo_type='dataset', repo_id='lmms-lab/LiveBench'), pr_revision=None, pr_num=None)